In [26]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer,Embedding,SpatialDropout1D
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import json

In [27]:
with open("config.json", "r") as f:
    config = json.load(f)

In [28]:
model_config = config["models"]
model_config

{'enabled': True,
 'model_name': 'Cases',
 'model': {'type': 'neural_net',
  'alpha': 1e-06,
  'hidden_layer_sizes': 5000,
  'learning_rate_init': 0.01,
  'max_iter': 100000,
  'batch_size': 32,
  'tol': 1e-06,
  'n_iter_no_change': 250,
  'verbose': False},
 'datagrabber_class': 'CasesDataGrabber',
 'grab_data_from_server': True,
 'offline_dataset_date': '2020-09-26',
 'days_to_predict': 10}

In [29]:
df = pd.read_csv('Confirmed_Cases_US_Vaccinated_NewDataset_copy.csv').drop(columns=['Unnamed: 0'])
df

,ObservationDate,Confirmed,People_Fully_Vaccinated
0,2020-12-20,17954675.0,0.0
1,2020-12-21,18153724.0,0.0
2,2020-12-22,18351735.0,0.0
3,2020-12-23,18581353.0,0.0
4,2020-12-24,18775557.0,0.0
...,...,...,...
129,2021-04-28,32230850.0,199426393.0
130,2021-04-29,32289049.0,202713598.0
131,2021-04-30,32346971.0,206220484.0
132,2021-05-01,32392274.0,210278776.0


In [30]:
## Use the data from April to May as test
df[df['ObservationDate']=='2021-04-01'].index

Int64Index([102], dtype='int64')

In [50]:
x = np.array(df.iloc[:102,:].index.values.tolist()).reshape(-1, 1)
x

array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
      

In [51]:
list = []
for i in range(0,32):
    list.append(102 +i)
x_pred = np.array(list).reshape(-1, 1)
x_pred

array([[102],
       [103],
       [104],
       [105],
       [106],
       [107],
       [108],
       [109],
       [110],
       [111],
       [112],
       [113],
       [114],
       [115],
       [116],
       [117],
       [118],
       [119],
       [120],
       [121],
       [122],
       [123],
       [124],
       [125],
       [126],
       [127],
       [128],
       [129],
       [130],
       [131],
       [132],
       [133]])

In [57]:
y_v_train = df.iloc[:102,-1]
y_v_train

0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
          ...     
97     102491174.0
98     105451667.0
99     107528783.0
100    109153459.0
101    111562791.0
Name: People_Fully_Vaccinated, Length: 102, dtype: float64

In [58]:
y_train = []

for elem in y_v_train:
    y_train_tmp = []
    y_train_tmp.append(elem)
    y_train.append(y_train_tmp)

y_train_np = np.array(y_train)
y_train_np

array([[0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [1.56688500e+06],
       [2.05742000e+06],
       [0.00000000e+00],
       [3.22104800e+06],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [4.04624800e+06],
       [4.32283800e+06],
       [4.78992200e+06],
       [5.51390600e+06],
       [6.05573000e+06],
       [6.43367200e+06],
       [6.69278000e+06],
       [6.96384200e+06],
       [7.60210600e+06],
       [8.52611200e+06],


In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y_train_np, test_size=0.1)

In [66]:
activation="tanh"
loss="binary_crossentropy"
metrics=['accuracy']
optimizer='adam'
epochs=10

In [67]:
TFmodel= Sequential()
TFmodel.add(Embedding(50000, 64, input_length=1))
TFmodel.add(SpatialDropout1D(0.2))
TFmodel.add(LSTM(100, dropout=0.3, recurrent_dropout=0))
TFmodel.add(Dense(11, activation=activation))
TFmodel.compile(loss=loss, metrics=metrics, optimizer=optimizer)

In [68]:
tensorboard = TensorBoard(log_dir='logs')
earlystopping = EarlyStopping(mode='min', patience=3, restore_best_weights=True)
history = TFmodel.fit(x_train,y_train,epochs=epochs, batch_size=32, callbacks=[tensorboard,earlystopping], validation_split=0.1)

Epoch 1/10


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_9/lstm_9/StatefulPartitionedCall]]
	 [[gradient_tape/sequential_9/embedding_9/embedding_lookup/Reshape/_38]] [Op:__inference_train_function_32892]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
y_v_pred

In [ ]:
plt.scatter(x, y_v_train, s=10)

In [ ]:
y_v_test = df.iloc[102:,-1]
y_v_test

In [ ]:
plt.plot(x_pred, y_v_pred, color='m', label="Predicted Vaccination")
plt.plot(x_pred, y_v_test, color='r',label="Real Vaccination")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("vaccination")
plt.show()

In [ ]:
y_c_train = df.iloc[:102,1]
vaccination = df.iloc[:102,-1]
vaccination

In [ ]:
x_c_train = np.vstack((np.array(df.iloc[:102,:].index.values.tolist()), vaccination)).T
x_c_train

In [ ]:
plt.scatter(x,y_c_train,s=10)

In [ ]:
neural_net_model = NeuralNetModel(model_config["model_name"])
neural_net_model.train(x_c_train, y_c_train, model_config["model"])

In [ ]:
x_c_test = np.vstack((np.array(list), y_v_pred)).T
x_c_test

In [ ]:
y_c_pred = neural_net_model.get_predictions(x_c_test)
y_c_pred

In [ ]:
x_c_test_real = np.vstack((np.array(list), np.array(y_v_test.tolist()))).T
x_c_test_real

In [ ]:
y_c_pred_real = neural_net_model.get_predictions(x_c_test_real)
y_c_pred_real

In [ ]:
y_c_test = df.iloc[102:,1]
y_c_test

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.plot(x_pred, y_c_test, color='r', label="Real confirmed cases")
plt.plot(x_pred,y_c_pred,color='b', label="predicted confirmed cases")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("Cases")

plt.subplot(1,2,2)
plt.plot(x_pred, y_c_test, color='r', label="Real confirmed cases")
plt.plot(x_pred,y_c_pred_real,color='b', label="predicted confirmed cases with real vaccination")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("Cases")

plt.show()

In [ ]:
plt.figure(figsize=(18,6))

plt.subplot(1,3,1)
plt.plot(x_pred, y_v_pred, color='r', label="predicted vaccination")
plt.plot(x_pred,y_c_pred,color='b', label="predicted confirmed cases")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("Cases/Vacciantions")

plt.subplot(1,3,2)
plt.plot(x_pred, y_v_test, color='r', label="real vaccination")
plt.plot(x_pred,y_c_test,color='b', label="real confirmed cases")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("Cases/Vacciantions")

plt.subplot(1,3,3)
plt.plot(x_pred, y_v_test, color='r', label="real vaccination")
plt.plot(x_pred,y_c_pred_real,color='b', label="predicted confirmed cases with real vaccination")
#plt.title("Amount of " + model_name + " in each day")
plt.legend()
plt.xlabel("Day")
plt.ylabel("Cases/Vacciantions")

plt.show()